<a href="https://colab.research.google.com/github/jurader/automatic_analysis_of_MALT_HE_image/blob/main/automatical_analysis_of_HE_image_for_MALT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/jurader/automatic_analysis_of_MALT_HE_image.git

Cloning into 'automatic_analysis_of_MALT_HE_image'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 29 (delta 10), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (29/29), 80.16 KiB | 1.41 MiB/s, done.
Resolving deltas: 100% (10/10), done.


# Demo image

The image in our Github repository is derived from the following paper.

https://www.hindawi.com/journals/crigm/2011/945752/


In [2]:
%cd automatic_analysis_of_MALT_HE_image/

/content/automatic_analysis_of_MALT_HE_image


In [3]:
import os

import numpy as np
import pandas as pd
import skimage


def run_all(args):
    df = pd.read_csv(args.i)
    N = len(df)

    results = []
    paths = df["filepath"]

    for i in range(N):
        i = paths[i]
        tim = skimage.io.imread(i)

        r = tim[:,:,0]
        g = tim[:,:,1]
        b = tim[:,:,2]
        sh = r.shape

        if args.tr is None:
            mask_r = np.ones(sh)
        else:
            mask_r = (r > args.tr).astype(np.uint8)

        if args.tg is None:
            mask_g = np.ones(sh)
        else:
            mask_g = np.logical_and(g>args.tg, g<args.tgg).astype(np.uint8)

        if args.tb is None:
            mask_b = np.ones(sh)
        elif args.tb == -1:
            mask_b = (r > b).astype(np.uint8)
        else:
            mask_b = (b > args.tb).astype(np.uint8)

        m = ((mask_r + mask_g + mask_b) > 2).astype(np.uint8)
        v = m.sum()

        results.append(v)
        print(v, i)

    final_df = pd.DataFrame(results)
    final_df["filepath"] = df["filepath"]

    path = "%s/%s.csv" % (args.o, sig)
    print("Saving CSV:", path)
    final_df.to_csv(path, header=True, index=False)

    return


######################################
# Main
######################################
if __name__ == "__main__":
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument("-i", type=str, default="./list.csv")
    parser.add_argument("-o", type=str, default="./result_csv/")
    parser.add_argument("--tr", type=int, default=200)
    parser.add_argument("--tg", type=int, default=175)
    parser.add_argument("--tgg", type=int, default=225)
    parser.add_argument("--tb", type=int, default=-1)

    args = parser.parse_args(args=[])
    csv = os.path.basename(args.i).replace(".", "_")
    args.sig = sig = "%s__tr%s_tg%s_tgg%s_tb%s" % (csv, args.tr, args.tg, args.tgg, args.tb)

    print(args)
    os.makedirs(args.o, exist_ok=True)

    run_all(args)



Namespace(i='./list.csv', o='./result_csv/', tr=200, tg=175, tgg=225, tb=-1, sig='list_csv__tr200_tg175_tgg225_tb-1')
364 MALT_from_paper.jpg
Saving CSV: ./result_csv//list_csv__tr200_tg175_tgg225_tb-1.csv
